# Zipline Coding Exercises

In this notebook you will create your own pipeline and create custom factors and filters for it. 

### Install Packages

In [1]:
!sed -Ei 's/^# deb-src /deb-src /' /etc/apt/sources.list 
!apt-get update 
!apt-get install libhdf5-dev -y

Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [99.8 kB]
Hit:2 http://archive.ubuntu.com/ubuntu xenial InRelease                        
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [99.8 kB]      
Get:5 http://archive.canonical.com/ubuntu xenial InRelease [11.5 kB]           
Get:6 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu xenial InRelease [23.8 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [97.4 kB]    
Get:8 https://cli-assets.heroku.com/apt ./ InRelease [2,550 B]                 
Get:9 http://security.ubuntu.com/ubuntu xenial-security/main Sources [311 kB]
Get:10 http://security.ubuntu.com/ubuntu xenial-security/restricted Sources [3,239 B]
Get:11 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [256 kB]
Get:12 http://security.ubuntu.com/ubuntu xenial-security/multiverse Sources [4,950 B]
Get:13 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [2,051 kB]
Get:14 http://

In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 1.3MB 8.4MB/s eta 0:00:01    92% |█████████████████████████████▋  | 1.2MB 27.7MB/s eta 0:00:01
    100% |████████████████████████████████| 13.4MB 2.5MB/s eta 0:00:01   26% |████████▋                       | 3.6MB 25.2MB/s eta 0:00:01    76% |████████████████████████▍       | 10.2MB 25.4MB/s eta 0:00:01
    100% |████████████████████████████████| 2.5MB 8.5MB/s eta 0:00:01    45% |██████████████▌                 | 1.1MB 26.6MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 16.6MB/s ta 0:00:01
    100% |████████████████████████████████| 26.3MB 1.3MB/s eta 0:00:01  8% |██▉                             | 2.3MB 24.4MB/s eta 0:00:01    13% |████▍                           | 3.6MB 25.7MB/s eta 0:00:01    18% |█████▉                          | 4.8MB 22.6MB/s eta 0:00:01    44% |██████████████▎                 | 11.7MB 24.6MB/s eta 0:00:01    49% |████████████████                | 13.1MB 20.7MB/s eta 0:00:01    53% |█████████████████▏   

# Load Data

In [3]:
import os

from zipline.data import bundles
from zipline.data.bundles.csvdir import csvdir_equities


# Specify the bundle name
bundle_name = 'm4-quiz-eod-quotemedia'

# Create an ingest function 
ingest_func = csvdir_equities(['daily'], bundle_name)

# Register the data bundle and its ingest function
bundles.register(bundle_name, ingest_func);

# Set environment variable 'ZIPLINE_ROOT' to the path where the most recent data is located
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..', 'data', 'module_4_quizzes_eod')

# Load the data bundle
bundle_data = bundles.load(bundle_name)

# TODO: Build an Empty Pipeline with a Screen

In the code below create an empty with a screen that filters the pipeline output for stocks with a 60-day average dollar volume greater than \$50,000.

In [6]:
# import resources
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume

# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length = 60) > 50000

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen=universe)


# TODO: Create a Pipeline Engine

In the code below, create a data loader that loads our data bundle using Zipline's `USEquityPricingLoader`. Then create a function to be passed to the `get_loader` parameter in the pipeline engine. Using the NYSE trading calendar create a pipeline engine using Zipline's `SimplePipelineEngine`

In [7]:
# import resources
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.loaders import USEquityPricingLoader

# Set the dataloader
pricing_loader = USEquityPricingLoader(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)

# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader

# Set the trading calendar
trading_calendar = get_calendar('NYSE')

# Create a Pipeline engine
engine = SimplePipelineEngine(get_loader = choose_loader,
                              calendar = trading_calendar.all_sessions,
                              asset_finder = bundle_data.asset_finder)

# TODO: Create Factors and Filters For The Pipeline

### Factors

In the code below, create two factors:

1. **Custom Factor**: Create a custom factor, `percent_difference`,  by combining a 30-day average closing price factor and a 60-average closing price factor. The `percent_difference` factor will calculate the difference between the 30-day and 60-average closing price factors, and will then normalize this difference by the 60-average closing price factor.


2. **Daily Returns Factor**: Create a factor that returns the daily percent change in closing price. To do this, use Zipline's built-in [DailyReturns](https://www.zipline.io/appendix.html#zipline.pipeline.factors.DailyReturns) factor.

Both factors should use the closing prices in the `USEquityPricing`  dataset as inputs. 




### Filters

In the code below, you will create a **Custom Filter**, `tradable_asset`, that returns `True` for the top 20 securities of average dollar volume in a 30 day-window that have a latest closing price above \$30. To do this, you will create two filters first and them combine them. For the first filter, you can use Zipline's built-in factor `AverageDollarVolume()` and its method `.top()` to select the top 20 assets of average dollar volume in a 30 day-window. To get the latest close price of the `USEquityPricing` dataset you can use `USEquityPricing.close.latest`. Finally you can combine these two filters by using the logical operator `&`. 

In [8]:
# import resources
import pandas as pd
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.factors import DailyReturns

# Create the 30-day average closing price factor
avg_close_30 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length=30)

# Create the 60-day average closing price factor
avg_close_60 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length=60)

# Create the custom factor
percent_difference = avg_close_30/avg_close_60 - 1

# Create the daily returns factor
daily_returns = DailyReturns(inputs = [USEquityPricing.close])

# Create a filter for the top 20 securities of average dollar volume in a 30 day-window
top_20 = AverageDollarVolume(window_length=30).top(20)

# Create a filter for the latest closing price above $30
above_30 = USEquityPricing.close.latest > 30

# Create a custum filter
custom_filter = top_20 & above_30

# TODO: Add Factors and Filters To The Pipeline

In the code below, add the factors and filters you created above to the pipeline

In [9]:
# Add the custom factor to the pipeline
pipeline.add(percent_difference, 'Percent Difference')

# Add the daily returns factor to the pipeline
pipeline.add(daily_returns, 'Daily Return')

# Add the custom filter to the pipeline
pipeline.add(custom_filter, 'Custom Filter')

# TODO: Run The Pipeline

In the code below, run the pipeline for the dates given

In [10]:
# Set starting and end dates
start_date = pd.Timestamp('2014-01-06', tz='utc')
end_date = pd.Timestamp('2016-01-05', tz='utc')

# Run our pipeline for the given start and end dates
pipe = engine.run_pipeline(pipeline, start_date, end_date)

# Display the pipeline output
pipe.head()

Custom Filter  Daily Return  \
2014-01-06 00:00:00+00:00 Equity(0 [A])             False      0.006951   
                          Equity(1 [AAL])           False      0.046523   
                          Equity(2 [AAP])           False      0.028611   
                          Equity(3 [AAPL])           True     -0.021972   
                          Equity(4 [ABBV])          False      0.006147   

                                            Percent Difference  
2014-01-06 00:00:00+00:00 Equity(0 [A])               0.025939  
                          Equity(1 [AAL])             0.055887  
                          Equity(2 [AAP])             0.051009  
                          Equity(3 [AAPL])            0.035906  
                          Equity(4 [ABBV])            0.033018

## Solution

[Solution notebook](zipline_coding_exercises_solution.ipynb)